Необходимые импорты

In [2]:
import requests
from bs4 import BeautifulSoup
import time

In [6]:
url = 'https://habr.com/ru/search/'

query_list = ['python', 'анализ данных', 'deepseek']

def get_posts(url, query_list):
    query_result_list = []
    for query in query_list:
        params = {
            'q': query,
            'target_types': 'posts',
            'order': 'relevance'
        }

        res = requests.get(url, params)
        time.sleep(0.2)
        soup = BeautifulSoup(res.text)
        query_result_list.append(soup)

        # Находим тег верхнего уровня
        top_level_tag = soup.find('div', class_='tm-pagination__pages')

        # Находим все вложенные теги и берем последний
        nested_tags = top_level_tag.find_all('div', class_='tm-pagination__page-group')
        last_nested_tag = nested_tags[-1] if nested_tags else None
        last_page = last_nested_tag.find_all('a', class_='tm-pagination__page')[-1].text

        print(last_page)
    return query_result_list

print(len(get_posts(url, query_list)))

50
50
13
3
